In [2]:
import pandas as pd
import time 
import warnings
from tqdm.notebook import trange, tqdm
from os import listdir
from os.path import isfile, join
import glob

warnings.filterwarnings('ignore')

In [26]:
vocab = {
    0 : 'Пн',
    1 : 'Вт',
    2 : 'Ср',
    3 : 'Чт',
    4 : 'Пт',
    5 : 'Сб',
    6 : 'Вс'
}


In [45]:
def create_target_set(target_name, how_to_shift):
    Target_KS = pd.read_csv(target_name+"P_in_out.csv",sep = ";")
    Target_KS["DateTime"] = pd.to_datetime(Target_KS["DateTime"])   
    Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
    abs = pd.DataFrame()
    abs["DateTime"] = pd.date_range(Target_KS["DateTime"].min(), periods=((Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).seconds/3600 + (Target_KS["DateTime"].max() - Target_KS["DateTime"].min()).days*24 +1), freq="h")
    Target_KS = pd.merge(abs,Target_KS,how='left',on="DateTime")
    Target_KS = Target_KS.ffill()
     #Тут будет заполнение пропусков чтобы дискретность была час в будущем
    if how_to_shift == "1":
        Target_KS.loc[Target_KS['dayofweek'] == 0,'DateTime'] -= pd.DateOffset(hours=72)   
        Target_KS.loc[Target_KS['dayofweek'] == 1,'DateTime'] -= pd.DateOffset(hours=96) 
        Target_KS.loc[Target_KS['dayofweek'] == 2,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 3,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 4,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 5,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS.loc[Target_KS['dayofweek'] == 6,'DateTime'] -= pd.DateOffset(hours=48)
        Target_KS["dayofweek"] = Target_KS["DateTime"].dt.dayofweek
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        Target_KS = Target_KS.drop(Target_KS[(Target_KS["DateTime"].dt.year < 2021)].index)
        return Target_KS
    else:        
        Target_KS["dayofweek"] = Target_KS["dayofweek"].replace(vocab, regex=True)
        Target_KS.columns = [f'{c}_'+ target_name if c!="DateTime" else c for c in Target_KS]
        return Target_KS
   
    

In [46]:
def prepare_dataset(target_name, how_to_shift_target):
    grs_list = glob.glob("ГРС/*")
    auto_plan_list = glob.glob("Автоплан/*")
    target_set = create_target_set(target_name, how_to_shift_target)
    schema_info = pd.read_csv(target_name+".csv",sep = ";")
    schema_info["DateTime"] = pd.to_datetime(schema_info["DateTime"])
    target_set = pd.merge(target_set,schema_info,how='left',on="DateTime")
    for grs in tqdm(grs_list):
        feature = pd.read_csv(grs, sep = ";")
        feature["DateTime"] = pd.to_datetime(feature["DateTime"])
        feature.columns = [f'{c}_'+ grs.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    for auto_list in tqdm(auto_plan_list):
        feature = pd.read_csv(auto_list, sep = ";")
        if "Date" in feature.columns:
            feature["Data"] = feature["Date"]
            feature.drop(columns=["Date"],inplace=True)
        try:
            feature["DateTime"] = pd.to_datetime(feature["Data"])
        except KeyError:
            continue
        feature.drop(columns=["Data"],inplace=True)
        feature.columns = [f'{c}_'+ auto_list.split("\\")[1][:-4] if c!="DateTime" else c for c in feature]
        target_set = pd.merge(target_set,feature,how='left',on="DateTime")
    #target_set = target_set.ffill()
    return target_set
        
    

In [47]:
t_set = prepare_dataset("КС-15", "1")

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [48]:
t_set

,DateTime,Object_КС-15,Pin_КС-15,Pout_КС-15,dayofweek_КС-15,Object,Tnv,GPAinwork,PNA,Schema1,...,82_КС-19 Факт по фидерам,83_КС-19 Факт по фидерам,84_КС-19 Факт по фидерам,85_КС-19 Факт по фидерам,86_КС-19 Факт по фидерам,87_КС-19 Факт по фидерам,88_КС-19 Факт по фидерам,89_КС-19 Факт по фидерам,ID_Кто планировал,Login_Кто планировал
0,2021-01-01 00:00:00,2010.0,"36,5","46,4",Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,14.0,Немытых
1,2021-01-01 01:00:00,2010.0,"36,5","46,4",Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
2,2021-01-01 02:00:00,2010.0,"36,5","46,7",Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,2021-01-01 03:00:00,2010.0,"36,5","46,7",Пт,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,NaN,NaN
4,2021-01-01 04:00:00,2010.0,"36,3","46,8",Пт,2010.0,-9.0,2.0,-6,21.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27143,2024-02-02 10:00:00,2010.0,"46,9","46,9",Пт,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27144,2024-02-02 11:00:00,2010.0,"46,9","46,9",Пт,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27145,2024-02-02 12:00:00,2010.0,"46,7","46,7",Пт,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27146,2024-02-02 13:00:00,2010.0,"46,7","46,7",Пт,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
t_set.to_excel("КС-15_set_v1.xlsx",index=None)